In this notebook, we will train a network to perform group composition on the abstract group S5, and then reverse engineer the algorithm learned. This notebook should be fairly self contained, and uses similar code to that in the paper, though slightly simplified. Full code for the paper is available at ...

# Setup


In [136]:
# Imports
import os
if "cd" not in globals():
    os.chdir("../")
    cd = True
import torch
import torch.nn as nn
import numpy as np
import math
import torch.nn.functional as F
from tqdm import tqdm
from transformer_lens.hook_points import HookPoint, HookedRootModule
from sympy.combinatorics.named_groups import SymmetricGroup as SymPySymmetricGroup
from sympy.combinatorics import Permutation, PermutationGroup
from utils.plotting import *

In [137]:
SEED = 2

Let's first create a class to contain all the information about our group. We include methods 
* to convert between permutations and their indices that we use to one-hot-encode them. 
* to compose permutations, based on their index
* to calculate the order of a permutation
* to calculate the signature of a permutation
* to cache all signatures of the group elements
* to calculate the inverse of a permutation
* to cache all inverses of group elements
* to calculate the full multiplication table of the group
* to return all pairs of permutations, and their product

In [138]:
class SymmetricGroup():
    """
    Class for the symmetric group of order index.
    """
    def __init__(self, index, init_all=False):
        """
        Initialise the group. Optionally calculate all other tensors needed to track metrics.

        Args:
            index (int): Index of group in family of symmetric groups.
            init_all (bool, optional): If false, only calculate what is required to train. If true, calculate all other tensors needed to track metrics. Defaults to False.
        """

        # build class on top of sympy
        self.G = SymPySymmetricGroup(index)
        self.index = index
        self.order = math.factorial(index)
        self.acronym = 'S'
        self.multiplication_table = self.compute_multiplication_table()
        
        # hacky method to find the index of the identity element 
        self.identity = [i for i in range(self.order) if self.idx_to_perm(i).order() == 1][0]
        
        self.inverses = self.compute_inverses()

        # compute the signatures of the elements
        self.signatures = self.compute_signatures()

        self.all_data, _ = self.get_all_data()
        self.all_data = self.all_data[:, :2]


    def idx_to_perm(self, x):
        """
        Convert an index to a permutation.

        Args:
            x (int): index of element in group

        Returns:
            Permutation: permutation object from sympy
        """
        return self.G._elements[x]

    def perm_to_idx(self, perm):
        """
        Converts a permutation to an index.

        Args:
            perm (Permutation): permutation object from sympy

        Returns:
            int: index of element in group
        """
        return self.G._elements.index(perm)

    def compose(self, x, y):
        """
        Compose elements of the group by converting to permutations, composing, and converting back.

        Args:
            x (int): left index
            y (int): right index

        Returns:
            int: index of composition
        """
        return self.perm_to_idx(self.idx_to_perm(x) * self.idx_to_perm(y))

    def perm_order(self, x):
        """
        Gets the order of a permutation.

        Args:
            x (int): index of element

        Returns:
            int: order of permutation
        """
        return self.idx_to_perm(x).order()

    def signature(self, x):
        """
        Gets the signature of a permutation.

        Args:
            x (int): index of element

        Returns:
            int: Integer \in {0, 1} representing the signature of the permutation.
        """
        return self.idx_to_perm(x).signature()
    
    def compute_signatures(self):
        """
        Compute and store the signature of each element in the group.

        Returns:
            torch.tensor: tensor of signatures
        """
        signatures = torch.tensor([self.signature(i) for i in range(self.order)]).cuda()
        return signatures
    
    def inverse(self, x):
        """
        Compute the inverse of an element of the group.

        Args:
            x (int): Index of element to inverse

        Returns:
            int: Index of inverse element
        """
        return (self.multiplication_table[x, :] == self.identity).nonzero().item()

    def compute_inverses(self):
        inverses = torch.zeros(self.order, dtype=torch.int64)
        for i in range(self.order):
            inverses[i] = self.inverse(i)
        return inverses
    
    def compute_multiplication_table(self):
        """
        Compute the multiplication table of the group. Caches/loads from file if possible.
        """
        print('Computing multiplication table...')
        table = torch.zeros((self.order, self.order), dtype=torch.int64).cuda()
        for i in tqdm(range(self.order)):
            for j in range(self.order):
                table[i, j] = self.compose(i, j)
        return table

    def get_all_data(self, shuffle_seed=False):
        """
        Get's all data and labels for the pairwise composition task.

        Args:
            shuffle_seed (bool, optional): Shuffle data for training. Defaults to False.

        Returns:
            torch.tensor: Tensor of shape (order*order, 3) where each row is (x, y, x*y).
        """
        data=torch.zeros((self.order*self.order, 3), dtype=torch.int64)
        shuffled_indices = None
        for i in range(self.order):
            for j in range(self.order):
                data[i*self.order+j, 0] = i
                data[i*self.order+j, 1] = j
                data[i*self.order+j, 2] = self.multiplication_table[i, j]
        if shuffle_seed:
            torch.manual_seed(shuffle_seed) 
            shuffled_indices = torch.randperm(self.order*self.order)
            data = data[shuffled_indices]
        return data.cuda(), shuffled_indices

In [139]:
group = SymmetricGroup(5)

Computing multiplication table...


100%|██████████| 120/120 [00:10<00:00, 11.46it/s]


The model architecture is an MLP as described in the paper. We subclass the HookedRootModule class from the transformer_lens library to easily cache activations on the forward pass.

In [140]:
model_cfg = {    
    "layers": {
        "embed_dim": 256,
        "hidden_dim": 128
    }
}

class OneLayerMLP(HookedRootModule):
    """ 
    A one layer MLP. W_x and W_y are embedding layers, whose outputs are concatenated and fed into a hidden layer. The result is unembedded by W_U.
    """
    def __init__(self, layers, n, seed=0):
        # embed_dim: dimension of the embedding
        # hidden : hidden dimension size
        # n : group order
        super().__init__()
        torch.manual_seed(seed)

        self.embed_dim = layers['embed_dim']
        hidden = layers['hidden_dim']

        # xavier initialise parameters
        self.W_x = nn.Parameter(torch.randn(n, self.embed_dim)/np.sqrt(self.embed_dim))
        self.W_y = nn.Parameter(torch.randn(n, self.embed_dim)/np.sqrt(self.embed_dim))
        self.W = nn.Parameter(torch.randn(2*self.embed_dim, hidden)/np.sqrt(2*self.embed_dim))
        self.relu = nn.ReLU()
        self.W_U = nn.Parameter(torch.randn(hidden, n)/np.sqrt(hidden))

        # hookpoints
        self.embed_stack = HookPoint()
        self.hidden = HookPoint()

        # We need to call the setup function of HookedRootModule to build an 
        # internal dictionary of modules and hooks, and to give each hook a name
        super().setup()

    def forward(self, data):
        x = data[:, 0] # (batch)
        half_x_embed = self.W_x[x] # (batch, embed_dim)
        y = data[:, 1] # (batch)
        half_y_embed = self.W_y[y] # (batch, embed_dim)
        embed_stack = self.embed_stack(torch.hstack((half_x_embed, half_y_embed))) # (batch, 2*embed_dim)
        hidden = self.hidden(self.relu(embed_stack @ self.W)) # (batch, hidden)
        out = hidden @ self.W_U # (batch, n)
        return out

def loss_fn(logits, labels):
    logits = logits.to(torch.float64)
    log_probs = logits.log_softmax(dim=-1)
    correct_log_probs = log_probs.gather(dim=-1, index=labels[:, None])[:, 0]
    return -correct_log_probs.mean()

def get_accuracy(logits, labels):
    """
    Compute accuracy of model.

    Args:
        logits (torch.tensor): (batch, group.order) tensor of logits
        labels (torch.tensor): (batch) tensor of labels

    Returns:
        float: accuracy
    """
    return ((logits.argmax(1)==labels).sum()/len(labels)).item()

model = OneLayerMLP(model_cfg['layers'], group.order).cuda()

To generate our train/test split, we just take a fixed proportion of the entire multiplication table of the group at random. We'll use a ratio of 0.4 here.

In [141]:
def generate_train_test_data(group, frac_train, seed=False):
    """
    Generate train and test data from a group's all data.
    """
    data, shuffled_indices = group.get_all_data(seed)
    train_size = int(frac_train*data.shape[0])
    train = data[:train_size]
    test = data[train_size:]
    train_data = train[:, :2]
    train_labels = train[:, 2]
    test_data = test[:, :2]
    test_labels = test[:, 2]
    return train_data, test_data, train_labels, test_labels, shuffled_indices



We'll checkpoint our models every 500 epochs, to later be able to analyse the model throughout training.

In [142]:
def save_checkpoint(model, epoch, task_dir, final=False):
    """
    Save model checkpoint.
    """
    path = f'{task_dir}/checkpoints/epoch_{epoch}.pt'
    if final:
        path = f'{task_dir}/model.pt'
    torch.save(model.state_dict(), path)

def load_checkpoint(model, task_dir, epoch=None, final=False):
    """
    Load model checkpoint.
    """
    path = f'{task_dir}/checkpoints/epoch_{epoch}.pt'
    if final:
        path = f'temp/model.pt'
    model.load_state_dict(torch.load(path), strict=False)
    return model

# Training

In [143]:
training_cfg = {
    "lr": 0.001,
    "betas": (0.9, 0.98),
    "weight_decay": 1,
    "num_epochs": 75000,
    "frac_train": 0.4,
}
train_data, test_data, train_labels, test_labels, shuffled_indices = generate_train_test_data(group, training_cfg["frac_train"], seed=SEED)
train_indices = shuffled_indices[:len(train_data)]
optimizer = torch.optim.AdamW(model.parameters(), lr=training_cfg["lr"], betas=training_cfg["betas"], weight_decay=training_cfg["weight_decay"])

# remove temp directory if it exists
if os.path.exists('temp'):
    os.system('rm -rf temp')

# create a temp directory
task_dir = f'temp'
if not os.path.exists(task_dir):
    os.makedirs(task_dir)

# create a temp directory for checkpoints
checkpoint_dir = f'temp/checkpoints'
if not os.path.exists(checkpoint_dir):
    os.makedirs(checkpoint_dir)

In [144]:
train_losses = []
test_losses = []
train_accs = []
test_accs = []

checkpoint_every = 500

for epoch in tqdm(range(training_cfg["num_epochs"])):
    train_logits = model(train_data)
    train_loss = loss_fn(train_logits, train_labels)
    train_loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    if epoch % checkpoint_every == 0:
        with torch.inference_mode():
            test_logits = model(test_data)
            test_loss = loss_fn(test_logits, test_labels)
            train_acc = get_accuracy(train_logits, train_labels)
            test_acc = get_accuracy(test_logits, test_labels)
            print(f"Epoch {epoch}: Train Loss: {train_loss}, Test Loss: {test_loss}, Train Accuracy: {train_acc}, Test Accuracy: {test_acc}")
        train_losses.append(train_loss.item())
        test_losses.append(test_loss.item())
        train_accs.append(train_acc)
        test_accs.append(test_acc)

        # save checkpoint
        save_checkpoint(model, epoch, task_dir)





  0%|          | 55/75000 [00:00<04:37, 270.09it/s]

Epoch 0: Train Loss: 4.788640449630546, Test Loss: 4.789846160405907, Train Accuracy: 0.008854166604578495, Test Accuracy: 0.007986110635101795


  1%|          | 545/75000 [00:01<03:46, 328.43it/s]

Epoch 500: Train Loss: 0.09221563697684555, Test Loss: 19.694246244583418, Train Accuracy: 1.0, Test Accuracy: 0.00011574073869269341


  1%|▏         | 1055/75000 [00:03<03:43, 330.48it/s]

...

 99%|█████████▊| 74061/75000 [03:43<00:02, 330.49it/s]

Epoch 74000: Train Loss: 1.5809986198968154e-06, Test Loss: 2.902854505176685e-06, Train Accuracy: 1.0, Test Accuracy: 1.0


 99%|█████████▉| 74537/75000 [03:45<00:01, 330.25it/s]

Epoch 74500: Train Loss: 1.5799228038793309e-06, Test Loss: 2.9033055499829177e-06, Train Accuracy: 1.0, Test Accuracy: 1.0


100%|██████████| 75000/75000 [03:46<00:00, 331.03it/s]


Let's plot loss and accuracy. We see the model groks! It initially memorises the training data, and much later on (around epoch 70k) suddenly generalises to the test set.

In [145]:
x = list(range(0, training_cfg["num_epochs"], checkpoint_every))
lines([train_losses, test_losses], x=x, labels=["Train Loss", "Test Loss"], xaxis="Epoch", yaxis="Loss", title="Loss vs Epoch", log_y=True)
lines([train_accs, test_accs], x=x, labels=["Train Accuracy", "Test Accuracy"], xaxis="Epoch", yaxis="Accuracy", title="Accuracy vs Epoch")

# Interpretability

Now let's try to interpret the model. To do so, we need to know some things about representation theory. Let's first create a class to hold information about representations. It will share some attributes with the group class. We will subclass from this class to create a class for each representation.

What information do we need, and how do we store it? 

Let $n$ be the order of group $G$. Each representation is a set $n$ $d \times d$ matrices. We store this in our representation objects as tensors of shape $n \times d^2$. 

We also compute what we call an 'orthogonal representation'. I'll explain why this is necessary later on.

Finally we compute the characters $\chi(abc^{-1})$ for each pair of inputs $a,b$ and output $c$. We store this in a tensor of shape $n \times n \times n$. This character tensor is used later to compute logit similarity.

## Setup: Defining Representations

In [146]:
class Representation():
    """
    Base class for all representations.
    """

    def __init__(self, compute_rep_params, index, order, multiplication_table, inverses, all_data, group_acronym, irrep=True):
        """
        Initialise the symmetric group representation.

        Args:
            compute_rep_params (tuple): representation specific parameters required for computing the representation
            index (int): group index in family
            order (int): order of the group
            multiplication_table (torch.tensor): square (group.order, group.order) tensor of group multiplication table 
            inverses (torch.tensor): vector of group inverses
            all_data ()
            irrep (Boolean)
        """

        self.index = index
        self.order = order
        self.multiplication_table = multiplication_table
        self.inverses = inverses
        self.all_data = all_data
        self.group_acronym = group_acronym

        # TODO: this is needed to get the dimension of generated representations - think up a better way of doing this
        self.compute_rep_params = compute_rep_params

        self.dim = self.get_rep_dim()

        self.rep = self.compute_rep(*compute_rep_params)

        if irrep:
            self.orth_rep = self.compute_orth_rep(self.rep)
            if self.friendly_name != 'trivial':
                self.logit_trace_tensor_cube = self.compute_logit_trace_tensor_cube()


    def get_rep_dim(self):
        return NotImplementedError

    def compute_rep(self):
        """
        Compute the representation. Must be implemented by child class.

        Raises:
            NotImplementedError
        """
        raise NotImplementedError
    
    def compute_orth_rep(self, rep):
        """
        Use QR decomposition to orthogonalise the representation but retain the subspace spanned by the columns. 

        Args:
            rep (torch.tensor): (group.order, dim^2) tensor of representation

        Returns:
            torch.tensor: (group.order, dim^2) tensor with orthonormal columns en
        """
        
        orth_rep = rep.reshape(self.order, self.dim * self.dim)
        orth_rep = torch.linalg.qr(orth_rep)[0]
        return orth_rep


    def compute_logit_trace_tensor_cube(self):
        """
        Under the hypothesis, the network computes tr(\rho(x)\rho(y)\rho(z^-1)) for some representation \rho.
        This function computes this trace tensor cube for a given representation
        
        Returns:
            torch.tensor: (group.order^3) trace tensor cube
        """
        print(f'Computing trace tensor cube for {self.friendly_name} representation')
        filename = f'utils/cache/{self.group_acronym}{self.index}/{self.group_acronym}{self.index}_{self.friendly_name}_trace_tensor_cube.pt'
        if os.path.exists(filename):
            print('... loading from file')
            t = torch.load(filename)
            return t 
        N = self.all_data.shape[0]
        t = torch.zeros((self.order*self.order, self.order), dtype=torch.float).cuda()
        for i in tqdm(range(N)):
            x = self.all_data[i, 0]
            y = self.all_data[i, 1]
            xy = self.multiplication_table[x, y]
            for z_idx in range(self.order):
                xyz = self.multiplication_table[xy, self.inverses[z_idx]]
                t[i, z_idx] = torch.trace(self.rep[xyz])
        t = t.reshape(self.order, self.order, self.order)
        f = open(filename, 'wb')
        torch.save(t, f)
        return t 

Now we construct all the irreducible representations of S5. One can find a full list [here](https://groupprops.subwiki.org/wiki/Linear_representation_theory_of_symmetric_group:S5). Feel free to skip over this construction if you're not interested in the details.

First, the boring trivial reprsentation. This is one dimensional - that is $d=1$, and all the elements map to the matrix $[1]$. We pass information needed to compute and initialise the representation from the child class to the parent class via `compute_rep_params` and `init_rep_params`.

### Constructing Irreps of S5


In [147]:
class TrivialRepresentation(Representation):
    """
    The trivial representation of the symmetric group.
    """
    def __init__(self, compute_rep_params, init_rep_params):
        """
        Initialise the trivial representation. 

        Args:
            compute_rep_params (list): idx_to_perm function required to compute the representation
            init_rep_params (dict): standard group parameters needed by the representation, including index, order, multiplication_table, inverses, all_data
        """
        self.friendly_name = 'trivial'
        super().__init__(compute_rep_params, **init_rep_params, irrep=True)
    
    def get_rep_dim(self):
        """
        Get the dimension of the representation.

        Returns:
            int: dimension of the representation
        """
        return 1

    def compute_rep(self):
        """
        Compute the trivial representation.

        Args:
            idx_to_perm (function): function to convert an index to a permutation

        Returns:
            torch.tensor: (group.order, 1) tensor of representation
        """
        return torch.ones((self.order, 1, 1), dtype=torch.float).cuda()
    

# we need some data from the group to initialise representations
rep_params = {
            'index': group.index,
            'order': group.order,
            'multiplication_table': group.multiplication_table,
            'inverses': group.inverses,
            'all_data': group.all_data,
            'group_acronym': group.acronym
        }

trivial_rep = TrivialRepresentation(compute_rep_params=[], init_rep_params=rep_params)

Next, the sign representation. This is also 1 dimensional, but the elements map to $[1]$ or $[-1]$ depending on whether the permutation is even or odd.

In [148]:
class SignRepresentation(Representation):
    """
    Initialise the sign representation of the symmetric group.

    """
    def __init__(self, compute_rep_params, init_rep_params):
        """
        Initialise the sign representation. 

        Args:
            compute_rep_params (list): list consisting of the signatures object required to compute the representation
            init_rep_params (dict): standard group parameters needed by the representation, including index, order, multiplication_table, inverses, all_data
        """
        self.friendly_name = 'sign'
        super().__init__(compute_rep_params, **init_rep_params)

    def get_rep_dim(self):
        """
        Get the dimension of the representation.

        Returns:
            int: dimension of the representation
        """
        return 1

    def compute_rep(self, signatures):
        """
        Compute the sign representation from the signatures.

        Args:
            signatures (torch.tensor): (group.order, 1) tensor of signatures

        Returns:
            torch.tensor: (group.order, 1, 1) tensor of sign representations
        """
        rep = torch.zeros(self.order, 1, 1).cuda()
        rep[:, 0, 0] = signatures
        return rep

sign_rep = SignRepresentation([group.signatures], rep_params)


Computing trace tensor cube for sign representation
... loading from file


Next, we construct the natural representation. This isn't actually an irreducible representation, but is easy to construct. We can later decompose it into a direct sum of the trivial representation and the `standard` representation. The natural representation itself is $d=5$, with group elements mapping naturally to the 5x5 [permutation matrices](https://en.wikipedia.org/wiki/Permutation_matrix) (with a 1 in each row and column, and 0s elsewhere).

In [149]:
class NaturalRepresentation(Representation):
    """
    Compute the natural representation of the symmetric group.
    """
    def __init__(self, compute_rep_params, init_rep_params):
        """
        Initialise the natural representation. 

        Args:
            compute_rep_params (list): idx_to_perm function required to compute the representation
            init_rep_params (dict): standard group parameters needed by the representation, including index, order, multiplication_table, inverses, all_data
        """
        self.friendly_name = 'natural'
        super().__init__(compute_rep_params, **init_rep_params, irrep=False)
    
    def get_rep_dim(self):
        """
        Get the dimension of the representation.

        Returns:
            int: dimension of the representation
        """
        return self.index

    def compute_rep(self, idx_to_perm):
        """
        Compute the natural representation by directly computing permutation matrices

        Args:
            idx_to_perm (function): Function that takes an index and returns the corresponding permutation object

        Returns:
            torch.tensor: (group.order, group.index, group.index) tensor of permutation matrices for each group element
        """
        idx = list(np.linspace(0, self.index-1, self.index))
        rep = torch.zeros(self.order, self.index, self.index).cuda()
        for x in range(self.order):
            rep[x, idx, idx_to_perm(x)(idx)] = 1
        return rep

natural_rep = NaturalRepresentation([group.idx_to_perm], rep_params)

/tmp/ipykernel_3087/3779721982.py:38: DeprecationWarning:

an integer is required (got type numpy.float64).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.



In [150]:
class StandardRepresentation(Representation):
    """
    Generate the standard representation of the symmetric group.

    """
    def __init__(self, compute_rep_params, init_rep_params):
        """
        Initialise the standard representation. 

        Args:
            compute_rep_params (list): list containing natural_reps object necessary to calculate the standard representation
            init_rep_params (dict): standard group parameters needed by the representation, including index, order, multiplication_table, inverses, all_data
        """
        self.friendly_name = 'standard'
        super().__init__(compute_rep_params, **init_rep_params)

    def get_rep_dim(self):
        """
        Get the dimension of the representation.

        Returns:
            int: dimension of the representation
        """
        return self.index-1
    
    def compute_rep(self, natural_reps):
        """
        Compute the standard representation from the natural representation.

        Args:
            natural_reps (torch.tensor): (group.order, group.index, group.index) tensor of natural representations (permutation matrices)

        Returns:
            torch.tensor: (group.order, group.index-1, group.index-1) tensor of standard representations
        """
        rep = []
        basis_transform = torch.zeros(self.index, self.index).cuda()
        for i in range(self.index-1):
            basis_transform[i, i] = 1
            basis_transform[i, i+1] = -1
        basis_transform[self.index-1, self.index-1] = 1 #to make the transform non singular
        for x in natural_reps:
            temp = basis_transform @ x @ basis_transform.inverse()
            rep.append(temp[:self.index-1, :self.index-1])
        rep = torch.stack(rep, dim=0).cuda()
        return rep  

standard_rep = StandardRepresentation([natural_rep.rep], rep_params)


Computing trace tensor cube for standard representation
... loading from file


In [151]:
class StandardSignRepresentation(Representation):
    def __init__(self, compute_rep_params, init_rep_params):
        """
        Initialise the tensor product of the standard and sign representation. 

        Args:
            compute_rep_params (list): list consisting of tensor of standard representation and signatures, required to compute the representation
            init_rep_params (dict): standard group parameters needed by the representation, including index, order, multiplication_table, inverses, all_data
        """
        self.friendly_name = 'standard_sign'
        super().__init__(compute_rep_params, **init_rep_params)

    def get_rep_dim(self):
        """
        Get the dimension of the representation.

        Returns:
            int: dimension of the representation
        """
        return self.index-1

    def compute_rep(self, standard_reps, signatures):
        """
        Compute the tensor product of the standard and sign representation.

        Args:
            standard_reps (torch.tensor): (group.order, group.index-1, group.index-1) tensor of standard representations
            signatures (torch.tensor): (group.order, 1) tensor of signatures

        Returns:
            torch.tensor: (group.order, group.index-1, group.index-1) tensor of standard_sign representations
        """
        rep = []
        for i in range(standard_reps.shape[0]):
            rep.append(signatures[i]*standard_reps[i])
        rep = torch.stack(rep, dim=0).cuda()
        return rep
    
standard_sign_rep = StandardSignRepresentation([standard_rep.rep, sign_rep.rep], rep_params)      

Computing trace tensor cube for standard_sign representation
... loading from file


In [152]:
class SymmetricRepresentationFromGenerators(Representation):
    def __init__(self, compute_rep_params, init_rep_params, name):
        """
        Initialise a representation of the symmetric group from a set of generators.

        Args:
            compute_rep_params (list): list consisting of the generators of the representation, sympy group object, and a function that maps indices to permutations
            init_rep_params (dict): standard group parameters needed by the representation, including index, order, multiplication_table, inverses, all_data
            name (str): name of the representation
        """
        self.friendly_name = name
        super().__init__(compute_rep_params, **init_rep_params)

    # TODO: make this less hacky
    def get_rep_dim(self):
        """
        Get the dimension of the representation.

        Returns:
            int: dimension of the representation
        """
        return list(self.compute_rep_params[0].values())[0].shape[0] # hacky way to get the dimension of the representation

    def compute_rep(self, generators, G, idx_to_perm):
        """
        Compute the representation from the generators.

        Args:
            generators (dict): dictionary of generators of the group along with their representations
            G (sympy group object): group object
            idx_to_perm (function): function that maps indices to permutations

        Returns:
            torch.tensor: (group.order, dim, dim) tensor of arbitrary representations
        """
        rep = torch.zeros(self.order, self.dim, self.dim).cuda()
        for i in range(self.order):
            generator_product = G.generator_product(idx_to_perm(i), original=True)
            result = torch.eye(self.dim).float()
            for g in generator_product:
                result = result @ generators[g]
            rep[i] = result
        return rep.cuda()


s5_5d_a_generators = {}
s5_5d_a_generators[Permutation(0, 1, 2, 3, 4)] = torch.tensor([
    [ 1, -1, -1,  1,  0],
    [ 0, -1, -1,  0,  1],
    [ 1, -1,  0,  0,  0],
    [ 0, -1,  0,  0,  0],
    [ 1, -1, -1,  0,  0]
]).float()
s5_5d_a_generators[Permutation(4, 3, 2, 1, 0)] = s5_5d_a_generators[Permutation(0, 1, 2, 3, 4)].inverse()
s5_5d_a_generators[Permutation(4)(0,1)] = torch.tensor([
    [ 0,  0, -1,  0,  0],
    [ 0,  0,  0, -1,  0],
    [-1,  0,  0,  0,  0],
    [ 0, -1,  0,  0,  0],
    [ 0,  0,  0,  0, -1]
]).float()
s5_5d_a_rep = SymmetricRepresentationFromGenerators([s5_5d_a_generators, group.G, group.idx_to_perm], rep_params, 's5_5d_a')

# 3,2 specht
s5_5d_b_generators = {}
s5_5d_b_generators[Permutation(0, 1, 2, 3, 4)] = torch.tensor([
    [-1,  1, -1,  0,  0],
    [ 0,  0,  0,  1, -1],
    [ 0,  0,  1,  0, -1],
    [ 1,  0,  0,  0,  0],
    [ 1,  0,  1,  0,  0]
]).float()
s5_5d_b_generators[Permutation(4, 3, 2, 1, 0)] = s5_5d_b_generators[Permutation(0, 1, 2, 3, 4)].inverse()
s5_5d_b_generators[Permutation(4)(0,1)] = torch.tensor([
    [ 1,  0,  0,  0,  0],
    [ 0,  0,  0, -1,  0],
    [-1,  0,  0,  0, -1],
    [ 0, -1,  0,  0,  0],
    [-1,  0, -1,  0,  0]
]).float()
s5_5d_b_rep = SymmetricRepresentationFromGenerators([s5_5d_b_generators, group.G, group.idx_to_perm], rep_params, 's5_5d_b')

# 3,1,1 specht
s5_6d_generators = {}
s5_6d_generators[Permutation(0, 1, 2, 3, 4)] = torch.tensor([
    [ 1, -1,  1,  0,  0,  0],
    [ 1,  0,  0, -1,  1,  0],
    [ 0,  1,  0, -1,  0,  1],
    [ 1,  0,  0,  0,  0,  0],
    [ 0,  1,  0,  0,  0,  0],
    [ 0,  0,  0,  1,  0,  0]
]).float()
s5_6d_generators[Permutation(4, 3, 2, 1, 0)] = s5_6d_generators[Permutation(0, 1, 2, 3, 4)].inverse()
s5_6d_generators[Permutation(4)(0,1)] = torch.tensor([
    [-1,  0,  0,  0,  0,  0],
    [ 0,  0,  0, -1,  0,  0],
    [ 0,  0,  0,  0, -1,  0],
    [ 0, -1,  0,  0,  0,  0],
    [ 0,  0, -1,  0,  0,  0],
    [ 0,  0,  0,  0,  0,  1]
]).float()
s5_6d_rep = SymmetricRepresentationFromGenerators([s5_6d_generators, group.G, group.idx_to_perm], rep_params, 's5_6d')


Computing trace tensor cube for s5_5d_a representation
... loading from file
Computing trace tensor cube for s5_5d_b representation
... loading from file
Computing trace tensor cube for s5_6d representation
... loading from file


In [153]:
group.irreps = {
    'trivial': trivial_rep,
    'sign': sign_rep,
    'standard': standard_rep,
    'standard_sign': standard_sign_rep,
    's5_5d_a': s5_5d_a_rep,
    's5_5d_b': s5_5d_b_rep,
    's5_6d': s5_6d_rep
}

group.non_trivial_irreps = group.irreps.copy()
del group.non_trivial_irreps['trivial']

Now we have all our representations loaded, let's inspect them...

## Reverse engineering the final model

In [154]:
all_data, _ = group.get_all_data(False)
all_data, all_labels = all_data[:, :2], all_data[:, 2]
with torch.inference_mode():
    all_logits, activations = model.run_with_cache(all_data)

    

In [155]:
print(all_labels.shape)

torch.Size([14400])


### Logit Similarity


In [156]:
def logit_trace_similarity(logits, trace_cube):
    """
    Compute cosine similarity between true logits and logits computed via tr(\rho(x)\rho(y)\rho(z^-1))

    Args:
        logits (torch.tensor): (batch, group.order) tensor of logits
        trace_cube (torch.tensor): (group.order, group.order, group.order) tensor of tr(\rho(x)\rho(y)\rho(z^-1))

    Returns:
        float: mean cosine similarity over batch
    """
    centered_logits = logits - logits.mean(dim=-1, keepdim=True)
    centered_logits = centered_logits.reshape(-1)
    trace = trace_cube.reshape(-1)
    sim = F.cosine_similarity(centered_logits, trace, dim=0)
    return sim


key_reps = []
percent_explained = 0
for rep_name, rep in group.non_trivial_irreps.items():
    sim = logit_trace_similarity(all_logits, rep.logit_trace_tensor_cube)
    if sim > 0.005:
        key_reps.append(rep_name)
        percent_explained += sim **2
    print(f'{rep_name}: {sim:.4f}')

# print key reps
print('\nKey reps:')
print(key_reps)

# print percent logit explained as a percentage
print('\nPercent logit explained:')
print(percent_explained.item() * 100)



sign: 0.5141
standard: 0.7714
standard_sign: 0.0000
s5_5d_a: 0.0000
s5_5d_b: 0.0007
s5_6d: 0.0000

Key reps:
['sign', 'standard']

Percent logit explained:
85.93581914901733


In [157]:
# visualise a slice of true logits, and key rep logits
all_logits_cube = all_logits.reshape(group.order, group.order, group.order)
true_logit_slice = all_logits_cube[:, :, 0]/all_logits_cube[:, :, 0].abs().max()
key_rep_logit_slices = {}
for key_rep in key_reps:
    key_rep_logit_slices[key_rep] = (group.irreps[key_rep].logit_trace_tensor_cube[:, :, 0]/group.irreps[key_rep].logit_trace_tensor_cube[:, :, 0].abs().max())

imshow(true_logit_slice, input1="a", input2="b", title="True logit c=0")
for key_rep, key_rep_logit_slice in key_rep_logit_slices.items():
    imshow(key_rep_logit_slice, input1="a", input2="b", title=f'{key_rep} logit c=0')
    


### Embed and Unembed

The model has no non linearity between it's embedding and hidden layer. It therefore makes sense to interpret the factored matrices together.

In [158]:
def get_embeds(model):
    """ 
    Get the embedding matrices for x and y
    """
    embeds = model.W_x @ model.W[:model.embed_dim, :], model.W_y @ model.W[model.embed_dim:, :]
    return embeds

def get_unembed(model):
    """ 
    Get the unembedding matrix
    """
    unembed = model.W_U
    return unembed

In [159]:
def percent_total_embed(model, orth_rep):
    """
    Compute the percent of the total embedding represented by the representation. Total embedding is the matmul of the embedding and the linear layer.

    Args:
        model (nn.Module): neural network
        orth_rep (torch.tensor): orthonormal representation

    Returns:
        (float, float): (total percent x, total percent y)
    """
    x_embed, y_embed = get_embeds(model)

    norm_x = x_embed.pow(2).sum()
    norm_y = y_embed.pow(2).sum()

    coefs_x = orth_rep.T @ x_embed
    coefs_y = orth_rep.T @ y_embed

    conts_x = coefs_x.pow(2).sum(-1) / norm_x
    conts_y = coefs_y.pow(2).sum(-1) / norm_y

    return conts_x.sum().item(), conts_y.sum().item()

def percent_unembed(model, orth_rep):
    """
    Compute the percent of the unembed represented by the representation.

    Args:
        model (nn.Module): neural network
        orth_rep (torch.tensor): orthonormal representation

    Returns:
        (float, float): (total percent x, total percent y)

    """
    W_U = get_unembed(model)
    norm_U = W_U.pow(2).sum()
    coefs_U = orth_rep.T @ W_U.T
    conts_U = coefs_U.pow(2).sum(-1) / norm_U
    return conts_U.sum().item()


In [160]:
print('Representation, frac left embed, frac right embed, frac unembed')
percent_left_explained_by_key_reps = 0
percent_right_explained_by_key_reps = 0
percent_unembed_explained_by_key_reps = 0
for rep_name, rep in group.non_trivial_irreps.items():
    left_embed, right_embed = percent_total_embed(model, rep.orth_rep)
    unembed = percent_unembed(model, rep.orth_rep)
    print(f'{rep_name}: {left_embed:.4f}, {right_embed:.4f}, {unembed:.4f}')

    if rep_name in key_reps:
        percent_left_explained_by_key_reps += left_embed
        percent_right_explained_by_key_reps += right_embed
        percent_unembed_explained_by_key_reps += unembed

print('\nFrac left explained by key reps:')
print(percent_left_explained_by_key_reps)
print('\nFrac right explained by key reps:')
print(percent_right_explained_by_key_reps)
print('\nFrac unembed explained by key reps:')
print(percent_unembed_explained_by_key_reps)

        

Representation, frac left embed, frac right embed, frac unembed
sign: 0.0673, 0.0674, 0.1000
standard: 0.9327, 0.9326, 0.8378
standard_sign: 0.0000, 0.0000, 0.0005
s5_5d_a: 0.0000, 0.0000, 0.0012
s5_5d_b: 0.0000, 0.0000, 0.0032
s5_6d: 0.0000, 0.0000, 0.0017

Frac left explained by key reps:
1.0000000149011612

Frac right explained by key reps:
1.000000037252903

Frac unembed explained by key reps:
0.9378509074449539


### Hidden Layer Neurons


In [161]:
for rep_name in group.irreps:
    group.irreps[rep_name].hidden_reps_x = group.irreps[rep_name].rep[all_data[:, 0]].reshape(group.order**2, -1)
    group.irreps[rep_name].hidden_reps_x_orth = torch.linalg.qr(group.irreps[rep_name].hidden_reps_x)[0]
    group.irreps[rep_name].hidden_reps_y = group.irreps[rep_name].rep[all_data[:, 1]].reshape(group.order**2, -1)
    group.irreps[rep_name].hidden_reps_y_orth = torch.linalg.qr(group.irreps[rep_name].hidden_reps_y)[0]
    group.irreps[rep_name].hidden_reps_xy = group.irreps[rep_name].rep[all_labels].reshape(group.order*group.order, -1)
    group.irreps[rep_name].hidden_reps_xy_orth = torch.linalg.qr(group.irreps[rep_name].hidden_reps_xy)[0]

In [162]:
def get_hidden(model):
    """ 
    Get the final MLP neuron activations for all data points
    """
    logits, activations = model.run_with_cache(all_data)
    hidden = activations['hidden'] 
    return hidden

def percent_hidden(model, rep_name):
    """
    Compute the percent of the total hidden representation represented by the representation matrices \rho(xy).

    Args:
        model (nn.Module): neural network
        hidden_reps_xy (torch.tensor): orthonormal hidden representations \rho(xy)

    """
    hidden = get_hidden(model)
    hidden = hidden - hidden.mean(dim=0, keepdim=True)

    hidden_norm = hidden.pow(2).sum()

    hidden_reps_x_orth = group.irreps[rep_name].hidden_reps_x_orth
    hidden_reps_y_orth = group.irreps[rep_name].hidden_reps_y_orth
    hidden_reps_xy_orth = group.irreps[rep_name].hidden_reps_xy_orth
    coefs_x = hidden_reps_x_orth.T @ hidden
    coefs_y = hidden_reps_y_orth.T @ hidden
    coefs_xy = hidden_reps_xy_orth.T @ hidden
    x_conts = coefs_x.pow(2).sum() / hidden_norm
    y_conts = coefs_y.pow(2).sum() / hidden_norm
    xy_conts = coefs_xy.pow(2).sum() / hidden_norm
    total_conts = x_conts + y_conts + xy_conts

    return x_conts.item(), y_conts.item(), xy_conts.item(), total_conts.item()

In [163]:
for rep in group.non_trivial_irreps:
    x_conts, y_conts, xy_conts, total_conts = percent_hidden(model, rep)
    print(f'{rep}: {x_conts:.4f}, {y_conts:.4f}, {xy_conts:.4f}, {total_conts:.4f}')
    

sign: 0.0244, 0.0244, 0.0244, 0.0732
standard: 0.3423, 0.3756, 0.1055, 0.8235
standard_sign: 0.0000, 0.0000, 0.0000, 0.0000
s5_5d_a: 0.0000, 0.0000, 0.0000, 0.0000
s5_5d_b: 0.0002, 0.0002, 0.0001, 0.0005
s5_6d: 0.0000, 0.0000, 0.0000, 0.0000


In [164]:
def hidden_to_logits(hidden, model):
    """ 
    Convert hidden activations to logits via the correct unembed
    """
    return hidden @ model.W_U


def hidden_excluded_and_restricted_loss(model, hidden_reps_xy_orth):
    """ 
    Restrict or exclude reps rho(ab) from the hidden layer and compute the loss on the restricted and excluded parts of the hidden layer.
    """
    hidden = get_hidden(model)
    
    coefs_xy = hidden_reps_xy_orth.T @ hidden
    hidden_xy = hidden_reps_xy_orth @ coefs_xy

    hidden_xy_restricted = hidden_xy
    hidden_xy_excluded = hidden - hidden_xy

    logits_restricted = hidden_to_logits(hidden_xy_restricted, model)
    logits_excluded = hidden_to_logits(hidden_xy_excluded, model)

    restricted_loss = loss_fn(logits_restricted, all_labels).item()
    excluded_loss = loss_fn(logits_excluded[train_indices], train_labels).item()

    return excluded_loss, restricted_loss

    
def total_hidden_excluded_and_restricted_loss(model, key_reps):
    """ 
    Restrict or exclude all key reps rho(ab) from the hidden layer and compute the loss on the restricted and excluded parts of the hidden layer.
    """
    hidden = get_hidden(model)

    hidden_restricted = torch.zeros_like(hidden)
    for rep_name in key_reps:
        coefs_xy =  group.irreps[rep_name].hidden_reps_xy_orth.T @ hidden
        hidden_xy = group.irreps[rep_name].hidden_reps_xy_orth @ coefs_xy
        hidden_restricted += hidden_xy

    hidden_excluded = hidden - hidden_restricted

    logits_restricted =  hidden_to_logits(hidden_restricted, model)
    logits_excluded =  hidden_to_logits(hidden_excluded, model)

    restricted_loss = loss_fn(logits_restricted, all_labels).item()
    excluded_loss = loss_fn(logits_excluded[train_indices], train_labels).item()

    return excluded_loss, restricted_loss

In [165]:
print('Excluded loss, restricted loss')
for rep in group.non_trivial_irreps:
    excluded_loss, restricted_loss = hidden_excluded_and_restricted_loss(model, group.irreps[rep].hidden_reps_xy_orth)
    print(f'{rep}: {excluded_loss:.4f}, {restricted_loss:.4f}')

excluded_loss, restricted_loss = total_hidden_excluded_and_restricted_loss(model, key_reps)
print(f'Total: {excluded_loss:.4f}, {restricted_loss:.4f}')
    

Excluded loss, restricted loss
sign: 0.0006, 4.0955
standard: 7.0111, 0.0001
standard_sign: 0.0000, 4.7875
s5_5d_a: 0.0000, 4.7875
s5_5d_b: 0.0000, 4.7614
s5_6d: 0.0000, 4.7875
Total: 7.3947, 0.0000


### Neuron Clustering


In [166]:
# evidence: neuron clustering pre ReLU

threshold = 1

x_embed, y_embed = get_embeds(model)

x_embed_summed = x_embed.pow(2).sum(dim=0)
off_neurons_x = (x_embed_summed < threshold).nonzero().squeeze()

y_embed_summed = y_embed.pow(2).sum(dim=0)
off_neurons_y = (y_embed_summed < threshold).nonzero().squeeze()

assert (off_neurons_x == off_neurons_y).all()

off_neurons = off_neurons_x

print(f'Off neurons: {len(off_neurons)}, {off_neurons}')

rep_neurons = {}

print('Neurons corresponding to each representation')
for rep_name in group.non_trivial_irreps:
    rep = group.irreps[rep_name].orth_rep
    coefs_x = rep.T @ x_embed
    coefs_y = rep.T @ y_embed
    coefs_x_summed = coefs_x.pow(2).sum(dim=0)
    coefs_y_summed = coefs_y.pow(2).sum(dim=0)

    x_neurons = (coefs_x_summed > threshold).nonzero().squeeze()
    y_neurons = (coefs_y_summed > threshold).nonzero().squeeze()
    assert (x_neurons == y_neurons).all()
    x_neurons = torch.tensor(x_neurons)
    if x_neurons.dim() == 0:
        x_neurons = x_neurons.unsqueeze(0)
    rep_neurons[rep_name] = x_neurons
    print(f'{rep_name}: {len(x_neurons)}, {x_neurons}')

all_neurons = torch.arange(model.W_U.shape[0])
unaccounted_neurons = set(all_neurons.tolist())
unaccounted_neurons -= set(off_neurons.tolist())
for rep_name, neurons in rep_neurons.items():
    unaccounted_neurons -= set(neurons.tolist())

print('Unaccounted neurons')
print(unaccounted_neurons)

Off neurons: 15, tensor([ 3,  5, 12, 17, 45, 50, 54, 59, 65, 68, 81, 87, 88, 95, 99],
       device='cuda:0')
Neurons corresponding to each representation
sign: 10, tensor([ 55,  62,  66,  75,  84,  89,  90, 102, 123, 127], device='cuda:0')
standard: 103, tensor([  0,   1,   2,   4,   6,   7,   8,   9,  10,  11,  13,  14,  15,  16,
         18,  19,  20,  21,  22,  23,  24,  25,  26,  27,  28,  29,  30,  31,
         32,  33,  34,  35,  36,  37,  38,  39,  40,  41,  42,  43,  44,  46,
         47,  48,  49,  51,  52,  53,  56,  57,  58,  60,  61,  63,  64,  67,
         69,  70,  71,  72,  73,  74,  76,  77,  78,  79,  80,  82,  83,  85,
         86,  91,  92,  93,  94,  96,  97,  98, 100, 101, 103, 104, 105, 106,
        107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120,
        121, 122, 124, 125, 126], device='cuda:0')
standard_sign: 0, tensor([], device='cuda:0', dtype=torch.int64)
s5_5d_a: 0, tensor([], device='cuda:0', dtype=torch.int64)
s5_5d_b: 0, tensor([], 

/tmp/ipykernel_3087/641860947.py:32: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).



In [167]:
# evidence and table: neuron clustering in post hidden layer

threshold = 110

hidden = activations['hidden'].reshape(group.order**2, -1)
hidden = hidden - hidden.mean(dim=0, keepdim=True)

hidden_summed = hidden.pow(2).sum(dim=0)
off_neurons = (hidden_summed < threshold).nonzero().squeeze()

assert (off_neurons == off_neurons_x).all()

print(f'Off neurons: {off_neurons}')


fracs_explained_x = {}
fracs_explained_y = {}
fracs_explained_xy = {}
fracs_explained_trivial = {}

for rep_name in key_reps:
    rep_x = group.irreps[rep_name].hidden_reps_x_orth
    rep_y = group.irreps[rep_name].hidden_reps_y_orth
    rep_xy = group.irreps[rep_name].hidden_reps_xy_orth

    trivial = group.irreps['trivial'].hidden_reps_x_orth

    coefs_x = rep_x.T @ hidden
    coefs_y = rep_y.T @ hidden
    coefs_xy = rep_xy.T @ hidden

    coefs_trivial = trivial.T @ hidden

    coefs_x_summed = coefs_x.pow(2).sum(dim=0)
    coefs_y_summed = coefs_y.pow(2).sum(dim=0)
    coefs_xy_summed = coefs_xy.pow(2).sum(dim=0)
    coefs_trivial_summed = coefs_trivial.pow(2).sum(dim=0)


    neurons = rep_neurons[rep_name]

    frac_x = (coefs_x_summed[neurons]).sum() / (hidden[:, neurons].pow(2).sum())
    frac_y = (coefs_y_summed[neurons]).sum() / (hidden[:, neurons].pow(2).sum())
    frac_xy = (coefs_xy_summed[neurons]).sum() / (hidden[:, neurons].pow(2).sum())
    frac_trivial = (coefs_trivial_summed[neurons]).sum() / (hidden[:, neurons].pow(2).sum())

    fracs_explained_x[rep_name] = frac_x
    fracs_explained_y[rep_name] = frac_y
    fracs_explained_xy[rep_name] = frac_xy
    fracs_explained_trivial[rep_name] = frac_trivial

print('Neurons corresponding to each representation')
for key in key_reps:
    print(f'frac variance explained in {key} x, y, xy: {fracs_explained_x[key], fracs_explained_y[key], fracs_explained_xy[key], fracs_explained_trivial[key]}')
    print(f'Sum of explained variance: {fracs_explained_x[key] + fracs_explained_y[key] + fracs_explained_xy[key] + fracs_explained_trivial[key]}')

Off neurons: tensor([ 3,  5, 12, 17, 45, 50, 54, 59, 65, 68, 81, 87, 88, 95, 99],
       device='cuda:0')
Neurons corresponding to each representation
frac variance explained in sign x, y, xy: (tensor(0.3333, device='cuda:0'), tensor(0.3334, device='cuda:0'), tensor(0.3333, device='cuda:0'), tensor(2.2360e-14, device='cuda:0'))
Sum of explained variance: 0.9999999403953552
frac variance explained in standard x, y, xy: (tensor(0.3694, device='cuda:0'), tensor(0.4053, device='cuda:0'), tensor(0.1138, device='cuda:0'), tensor(8.3030e-15, device='cuda:0'))
Sum of explained variance: 0.8885210156440735


### Extracting the final linear map in the representation basis

In [168]:
def projection_matrix_general(B):
    """Compute the projection matrix onto the space spanned by the columns of `B`
    Args:
        B: ndarray of dimension (D, M), the basis for the subspace
    
    Returns:
        P: the projection matrix
    """
    P = B @ (B.T @ B).inverse() @ B.T
    return P

hidden = activations['hidden'].reshape(group.order*group.order, -1)
hidden_to_reps_proj = {}
coefs = {}

for rep_name in key_reps:
    hidden_reps_xy = group.irreps[rep_name].hidden_reps_xy



    P = projection_matrix_general(hidden_reps_xy)
    hidden_xy = P @ hidden

    hidden_to_reps_proj[rep_name] = hidden_reps_xy.T @ hidden_xy
    #hidden_to_reps_proj[rep_name] = hidden_to_reps_proj[rep_name] / hidden_to_reps_proj[rep_name].norm(dim=1, keepdim=True)
    if rep_name == 'standard':
        plot = hidden_to_reps_proj[rep_name]
        plot = plot/plot.norm(dim=1, keepdim=True)
        plot = plot.detach().cpu().numpy() 
        fig = px.imshow(plot, color_continuous_scale='RdBu', color_continuous_midpoint=0.0, labels={'x':'neuron basis', 'y':'rep basis'})
        fig.update_layout(
            height = 180,
            margin = dict(l=80, r=80, t=0, b=30, pad=0)
        )
        # set the length of the colorbar to 1
        fig.update_layout(coloraxis_colorbar=dict(
            len  = 0.72,
        ))
        fig.show()

    hidden_in_rep = hidden_xy @ hidden_to_reps_proj[rep_name].T

    theoretical_reps = hidden_reps_xy.reshape(group.order*group.order, -1)

    hidden_in_rep_norm = hidden_in_rep.flatten() / hidden_in_rep.flatten().norm()
    theoretical_reps_norm = theoretical_reps.flatten() / theoretical_reps.flatten().norm()

    # MSE loss between hidden_in_rep and theoretical_reps
    sim = F.mse_loss(hidden_in_rep_norm, theoretical_reps_norm)
    print(f'MSE Loss between hidden layer and theoretical representations: {sim}')

    # get the coef
    coef = (hidden_in_rep.norm() / theoretical_reps.norm())
    coefs[rep_name] = coef

MSE Loss between hidden layer and theoretical representations: 0.0


MSE Loss between hidden layer and theoretical representations: 8.991650979339738e-09


In [169]:
# evidence: 
rep_name = 'standard'
W_U = model.W_U
rep = group.irreps[rep_name].rep.reshape(group.order, -1)
#rep = rep / rep.norm(dim=0, keepdim=True)
W_U_rep = hidden_to_reps_proj[rep_name] @ W_U @ rep [group.inverses]
fig = px.imshow(to_numpy(W_U_rep), color_continuous_scale='RdBu', color_continuous_midpoint=0.0, labels={'x':'$\Large \\rho(c^{-1})$', 'y':'$\Large \\rho(ab)$'})
fig.update_layout(
            margin = dict(l=50, r=80, t=20, b=80, pad=0),
            width = 500,
        )
        # make the colorbar length 1
fig.update_layout(coloraxis_colorbar=dict(
    lenmode="fraction",
    len=1.07,
))
fig.show()

real_linear_map = (W_U_rep > 1e5).float()
real_linear_map_norm = real_linear_map.flatten() / real_linear_map.flatten().norm()
W_U_rep_norm = W_U_rep.flatten() / W_U_rep.flatten().norm()
sim = F.mse_loss(real_linear_map_norm, W_U_rep_norm)
#sim = F.cosine_similarity(W_U_rep.flatten(), real_linear_map.flatten(), dim=0)
print(f'MSE loss between unembedding matrix and real linear map: {sim}')

MSE loss between unembedding matrix and real linear map: 1.525467450846918e-05


## Analysis during training


In [179]:
def sum_of_squared_weights(model):
    """
    Compute the sum of squared weights on the whole model.

    Args:
        model (nn.Module)

    Returns:
        float: sum of squared weights on entire group
    """

    sum_of_square_weights = 0

    if model.__class__.__name__ == "OneLayerMLP":
        sum_of_square_weights += torch.sum(model.W_x**2)
        sum_of_square_weights += torch.sum(model.W_y**2)
        sum_of_square_weights += torch.sum(model.W_U**2)
        sum_of_square_weights += torch.sum(model.W**2)
    
    return sum_of_square_weights.item()

In [180]:
checkpointed_epochs = list(range(0, training_cfg['num_epochs'], checkpoint_every))
metrics = {}
for rep in group.non_trivial_irreps:
    metrics['logit_trace_similarity_' + rep + '_rep'] = []
    metrics['percent_left_embed_' + rep + '_rep'] = []
    metrics['percent_right_embed_' + rep + '_rep'] = []
    metrics['percent_unembed_' + rep + '_rep'] = []
    metrics['percent_hidden_' + rep + '_rep'] = []
    metrics['excluded_loss_' + rep + '_rep'] = []
    metrics['restricted_loss_' + rep + '_rep'] = []
metrics['total_excluded_loss'] = []
metrics['total_restricted_loss'] = []
metrics['test_loss'] = []
metrics['test_acc'] = []
metrics['train_loss'] = []
metrics['train_acc'] = []
metrics['sum_of_square_weights'] = []
    
    
for epoch in checkpointed_epochs:
    load_checkpoint(model, task_dir, epoch)
    model.eval()
    metrics['test_loss'].append(loss_fn(model(test_data), test_labels).item())
    metrics['test_acc'].append(get_accuracy(model(test_data), test_labels))
    metrics['train_loss'].append(loss_fn(model(train_data), train_labels).item())
    metrics['train_acc'].append(get_accuracy(model(train_data), train_labels))
    all_logits = model(all_data)
    for rep in group.non_trivial_irreps:
        #metrics['logit_trace_similarity_' + rep + '_rep'].append(logit_trace_similarity(all_logits, group.irreps[rep].logit_trace_tensor_cube))
        left_embed, right_embed = percent_total_embed(model, group.irreps[rep].orth_rep)
        metrics['percent_left_embed_' + rep + '_rep'].append(left_embed)
        metrics['percent_right_embed_' + rep + '_rep'].append(right_embed)
        unembed = percent_unembed(model, group.irreps[rep].orth_rep)
        metrics['percent_unembed_' + rep + '_rep'].append(unembed)
        _, _, hidden_xy, _ = percent_hidden(model, rep)
        metrics['percent_hidden_' + rep + '_rep'].append(hidden_xy)
        excluded_loss, restricted_loss = hidden_excluded_and_restricted_loss(model, group.irreps[rep].hidden_reps_xy_orth)
        metrics['excluded_loss_' + rep + '_rep'].append(excluded_loss)
        metrics['restricted_loss_' + rep + '_rep'].append(restricted_loss)
    total_excluded_loss, total_restricted_loss = total_hidden_excluded_and_restricted_loss(model, key_reps)
    metrics['total_excluded_loss'].append(total_excluded_loss)
    metrics['total_restricted_loss'].append(total_restricted_loss)
    metrics['sum_of_square_weights'].append(sum_of_squared_weights(model))
    

    


In [171]:
non_trivial_irreps_list = list(group.non_trivial_irreps)

In [172]:
# plot left embeds
lines([metrics['percent_left_embed_' + rep + '_rep'] for rep in group.non_trivial_irreps],
      x=checkpointed_epochs,
      labels=non_trivial_irreps_list,
      title='Percent of left embedding matrix explained by rep',
      xaxis='epoch',
      yaxis='percent')

In [173]:
# plot right embeds
lines([metrics['percent_right_embed_' + rep + '_rep'] for rep in group.non_trivial_irreps],
      x=checkpointed_epochs,
      labels=non_trivial_irreps_list,
      title='Percent of right embedding matrix explained by rep',
      xaxis='epoch',
      yaxis='percent')

In [174]:
# plot unembeds
lines([metrics['percent_unembed_' + rep + '_rep'] for rep in group.non_trivial_irreps],
        x=checkpointed_epochs,
        labels=non_trivial_irreps_list,
        title='Percent of unembedding matrix explained by rep',
        xaxis='epoch',
        yaxis='percent')


In [175]:
print(metrics['percent_hidden_standard_rep'])

[0.0001500012876931578, 0.00047078728675842285, 0.0007364900666289032, 0.0010661304695531726, 0.0014370688004419208, 0.00184731581248343, 0.0023502500262111425, 0.002759685041382909, 0.0030899858102202415, 0.0033705136738717556, 0.003631064435467124, 0.003825458465144038, 0.0039922213181853294, 0.004171977750957012, 0.004311450757086277, 0.004425874445587397, 0.004525103606283665, 0.004686822649091482, 0.004805736243724823, 0.004941439256072044, 0.005065194796770811, 0.005187405273318291, 0.0053070466965436935, 0.005434718914330006, 0.0055747260339558125, 0.005723624024540186, 0.005867467727512121, 0.006032347213476896, 0.0062080854550004005, 0.006424125283956528, 0.006718774791806936, 0.007105672266334295, 0.007517695892602205, 0.007966691628098488, 0.008465652354061604, 0.009099755436182022, 0.009620602242648602, 0.010006606578826904, 0.010300724767148495, 0.010520825162529945, 0.010710221715271473, 0.010870308615267277, 0.01101385336369276, 0.011154270730912685, 0.011292294599115849

In [176]:
# percent hidden
lines([metrics['percent_hidden_' + rep + '_rep'] for rep in group.non_trivial_irreps],
        x=checkpointed_epochs,
        labels=non_trivial_irreps_list,
        title='Percent of hidden explained by rep',
        xaxis='epoch',
        yaxis='percent')


In [177]:
# plot losses
lines([metrics['train_loss'], metrics['test_loss'], metrics['total_excluded_loss'], metrics['total_restricted_loss']], x=checkpointed_epochs, log_y=True, labels=['train loss', 'test loss', 'total excluded loss', 'total restricted loss'], title='Losses', xaxis='Epoch', yaxis='Loss')

In [183]:
# excluded loss by rep
lines([metrics['excluded_loss_' + rep + '_rep'] for rep in group.non_trivial_irreps],
        x=checkpointed_epochs,
        labels=non_trivial_irreps_list,
        title='Excluded loss by rep',
        xaxis='epoch',
        yaxis='loss',
        log_y=True)

# restricted loss by rep
lines([metrics['restricted_loss_' + rep + '_rep'] for rep in group.non_trivial_irreps],
        x=checkpointed_epochs,
        labels=non_trivial_irreps_list,
        title='Restricted loss by rep',
        xaxis='epoch',
        yaxis='loss',
        log_y=True)



In [181]:
# plot sum of squared weights

lines([metrics['sum_of_square_weights']], x=checkpointed_epochs, log_y=True, labels=['sum of squared weights'], title='Sum of squared weights', xaxis='Epoch', yaxis='Sum of squared weights')